首先，我们将了解 PyTorch Geometric 如何将图存储为 PyTorch 张量。

然后，我们将使用 ogb 包加载和检查其中一个 Open Graph Benchmark (OGB) 数据集。OGB 是用于图机器学习的现实、大规模和多样化的基准数据集的集合。ogb 包不仅为每个数据集提供数据加载器，还提供模型评估器。

最后，我们将使用 PyTorch Geometric 构建我们自己的 GNN。然后，我们将在 OGB 节点属性预测和图形属性预测任务上训练和评估我们的模型。

注意：确保按顺序运行每个部分中的所有单元，以便中间变量/包将延续到下一个单元 完成本次实验的时间约为两小时

# 环境搭建

In [ ]:
import os

import torch
print("PyTorch has version {}".format(torch.__version__))

下载 PyG 的依赖，确保其与 torch 下载的版本契合，如果有问题可以查阅文档 [PyG's page](https://www.google.com/url?q=https%3A%2F%2Fpytorch-geometric.readthedocs.io%2Fen%2Flatest%2Fnotes%2Finstallation.html)

In [ ]:
# 安装 torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install ogb

# 1)PyG (数据集和数据)

PyTorch Geometric 有两个用于存储和/或将图转换为张量格式的类。
一个是 `torch_geometric.datasets`，它包含了各种常见的图数据集；
另一个是 `torch_geometric.data`，它提供了将图转换为 PyTorch 张量的相关数据处理功能。

在本节中，我们将学习如何将 `torch_geometric.datasets` 和 `torch_geometric.data` 结合使用。

## PyG 数据集

`torch_geometric.datasets` 类有许多图数据集，我们使用其一来探索其用法

In [ ]:
from torch_geometric.datasets import TUDataset

if 'IS_GRADESCOPE_ENV' not in os.environ:
  root = './enzymes'
  name = 'ENZYMES'

  # ENZYMES(酶)数据集
  pyg_dataset= TUDataset(root, name)

  # 其中有六百个图
  print(pyg_dataset)

### Question1: ENZYMES 数据集中有多少类，多少特征

In [ ]:
def get_num_classes(pyg_dataset):
  # TODO: 实现一个函数，接收一个 PyG 数据集对象，
  # 并返回该数据集的类别数量。

  num_classes =

  ############# Your code here ############
  ## (~1 行代码)
  ## 注意：
  ## 1. 自动补全功能可能会很有帮助。

  #########################################

  return num_classes

def get_num_features(pyg_dataset):
  # TODO: 实现一个函数，接收一个 PyG 数据集对象，
  # 并返回该数据集的特征数量。

  num_features =

  ############# Your code here ############
  ## (~1 行代码)
  ## 注意：
  ## 1. 自动补全功能可能会很有帮助。

  #########################################

  return num_features

if 'IS_GRADESCOPE_ENV' not in os.environ:
  num_classes = get_num_classes(pyg_dataset)
  num_features = get_num_features(pyg_dataset)
  print("{} dataset has {} classes".format(name, num_classes))
  print("{} dataset has {} features".format(name, num_features))


## PyG 数据

每个 PyG 数据集都存储了一个由 `torch_geometric.data.Data` 对象组成的列表，其中每个 `torch_geometric.data.Data` 对象表示一张图。

我们可以通过索引数据集获取 `Data` 对象。 
关于 `Data` 对象中包含哪些信息等更多内容，请参考[官方文档](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data)。

### Question 2： ENZYMES 数据集中 index 为 100 的图的 label 是什么？

In [ ]:
def get_graph_class(pyg_dataset, idx):
  # TODO: 实现一个函数，接收一个 PyG 数据集对象，
  # 和一个图在数据集中的索引，返回该图的类别/标签（为一个整数）。

  label = -1

  ############# Your code here ############
  ## (~1 行代码)

  #########################################

  return label

# 此处的 pyg_dataset 是用于图分类的数据集
if 'IS_GRADESCOPE_ENV' not in os.environ:
  graph_0 = pyg_dataset[0]
  print(graph_0)
  idx = 100
  label = get_graph_class(pyg_dataset, idx)
  print('Graph with index {} has label {}'.format(idx, label))


### Question 3：index 为 200 的图有多少条边？

In [ ]:
def get_graph_num_edges(pyg_dataset, idx):
  # TODO: 实现一个函数，接收一个 PyG 数据集对象，
  # 和该数据集中某个图的索引，返回该图中的边数（整数）。
  # 如果图是无向的，不应该重复计数边。
  # 例如，在一个无向图 G 中，若两个节点 v 和 u 之间有一条边，
  # 那么这条边只应该被计数一次。

  num_edges = 0

  ############# Your code here ############
  ## 注意：
  ## 1. 不能直接返回 data.num_edges
  ## 2. 我们假设图是无向的
  ## 3. 可以查看 PyG 数据集中自带的函数
  ## （大约 4 行代码）

  #########################################

  return num_edges

if 'IS_GRADESCOPE_ENV' not in os.environ:
  idx = 200
  num_edges = get_graph_num_edges(pyg_dataset, idx)
  print('Graph with index {} has {} edges'.format(idx, num_edges))


# 2) Open Graph Benchmark(OGB)

**Open Graph Benchmark（OGB）** 是一个用于图机器学习的现实、大规模且多样化的基准数据集集合。

这些数据集可以通过 OGB 的数据加载器（OGB Data Loader）**自动下载、处理并划分**。

随后，可以使用 OGB 的评估器（OGB Evaluator）以统一的方式对模型性能进行评估。

如果数据集自动下载速度较慢，可以从Nju Box下载：https://box.nju.edu.cn/d/5f1c0015382643c9be0d/

## 数据集和数据

OGB 也支持 PyG 的数据集/数据的类。此处我们查看 `ogbn-arxiv` 数据集

In [ ]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset



if 'IS_GRADESCOPE_ENV' not in os.environ:
  dataset_name = 'ogbn-arxiv'
  # 加载数据集并转换为稀疏图
  dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
  print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

  # 分离一张图出来
  data = dataset[0]
  print(data)

### Question 4: ogbn-arxiv 的图中有多少特征？

In [ ]:
def graph_num_features(data):
  # TODO: 实现一个函数，接收一个 PyG 的 data 对象，
  # 并返回该图的特征数量（为一个整数）。

  num_features = 0

  ############# Your code here ############
  ## (~1 行代码)

  #########################################

  return num_features

if 'IS_GRADESCOPE_ENV' not in os.environ:
  num_features = graph_num_features(data)
  print('The graph has {} features'.format(num_features))


# 3） GNN：节点属性预测

在本节中，我们将使用 PyTorch Geometric 构建第一个图神经网络。然后，我们会将其应用于**节点属性预测（节点分类）**任务。

具体来说，我们将以 **GCN（图卷积网络）** 作为图神经网络的基础（参考 [Kipf 等人, 2017](https://arxiv.org/abs/1609.02907)）。  
为此，我们将使用 PyG 内置的 `GCNConv` 层。

## 环境搭建

In [ ]:
import torch
import pandas as pd
import torch.nn.functional as F
print(torch.__version__)

# 使用 PyG 内建的 GCNConv
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

## 加载并处理数据

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  dataset_name = 'ogbn-arxiv'
  dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.Compose([T.ToUndirected(),T.ToSparseTensor()]))
  data = dataset[0]

  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # 如果你在使用 gpu ， device 应该是 cuda
  print('Device: {}'.format(device))

  data = data.to(device)
  split_idx = dataset.get_idx_split()
  train_idx = split_idx['train'].to(device)

## GCN 模型

现在我们来实现我们的 GCN 模型！

请根据下图所示的结构来实现 `forward` 函数：
![GCN 模型结构图](https://drive.google.com/uc?id=128AuYAXNXGg7PIhJJ7e420DoPWKb-RtL)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # TODO: 实现一个函数来初始化 self.convs、self.bns 和 self.softmax。

        super(GCN, self).__init__()

        # 一个包含 GCNConv 层的列表
        self.convs = None

        # 一个包含一维批归一化层（BatchNorm1d）的列表
        self.bns = None

        # log softmax 层
        self.softmax = None

        ############# Your code here ############
        ## 注意：
        ## 1. self.convs 和 self.bns 应该使用 torch.nn.ModuleList
        ## 2. self.convs 应包含 num_layers 个 GCNConv 层
        ## 3. self.bns 应包含 num_layers - 1 个 BatchNorm1d 层
        ## 4. self.softmax 应使用 torch.nn.LogSoftmax
        ## 5. GCNConv 需要设置的参数包括 'in_channels' 和 'out_channels'
        ##    更多信息请参考文档：
        ##    https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. BatchNorm1d 只需要设置 'num_features'
        ##    更多信息请参考文档：
        ##    https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html
        ## （大约 10 行代码）

        #########################################

        # 元素被置为 0 的概率（Dropout 概率）
        self.dropout = dropout

        # 是否跳过分类层并返回节点嵌入
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        # TODO: 实现一个函数，接收特征张量 x 和边索引张量 adj_t，
        # 并按结构图所示返回输出张量。

        out = None

        ############# Your code here ############
        ## 注意：
        ## 1. 按照结构图构建神经网络
        ## 2. 可以使用 torch.nn.functional.relu 和 torch.nn.functional.dropout
        ##    文档参考：https://pytorch.org/docs/stable/nn.functional.html
        ## 3. 不要忘了将 F.dropout 的 training 参数设置为 self.training
        ## 4. 如果 return_embeds 为 True，则跳过最后的 softmax 层
        ## （大约 7 行代码）

        #########################################

        return out


In [ ]:
def train(model, data, train_idx, optimizer, loss_fn):
    # TODO: 实现一个使用给定的优化器和损失函数训练模型的函数。
    model.train()
    loss = 0

    ############# Your code here ############
    ## 注意：
    ## 1. 对优化器执行 zero grad（清除梯度）
    ## 2. 将数据输入模型
    ## 3. 使用 train_idx 对模型输出和标签进行切片
    ## 4. 将切片后的输出和标签输入损失函数 loss_fn
    ## （大约 4 行代码）

    #########################################

    loss.backward()
    optimizer.step()

    return loss.item()


In [ ]:
# 测试函数
@torch.no_grad()
def test(model, data, split_idx, evaluator, save_model_results=False):
    # TODO: 实现一个使用给定的 split_idx 和 evaluator 来测试模型的函数。
    model.eval()

    # 模型在所有数据上的输出
    out = None

    ############# Your code here ############
    ## （大约 1 行代码）
    ## 注意：
    ## 1. 此处不进行索引切片

    #########################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    if save_model_results:
      print ("Saving Model Predictions")

      data = {}
      data['y_pred'] = y_pred.view(-1).cpu().detach().numpy()

      df = pd.DataFrame(data=data)
      # 本地保存为 CSV 文件
      df.to_csv('ogbn-arxiv_node.csv', sep=',', index=False)

    return train_acc, valid_acc, test_acc


In [ ]:
# 请不要改变 args
if 'IS_GRADESCOPE_ENV' not in os.environ:
  args = {
      'device': device,
      'num_layers': 3,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 100,
  }
  args

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  model = GCN(data.num_features, args['hidden_dim'],
              dataset.num_classes, args['num_layers'],
              args['dropout']).to(device)
  evaluator = Evaluator(name='ogbn-arxiv')

In [ ]:
# 请不要改变 args
# 使用 GPU 训练应该小于 10 分钟
import copy
if 'IS_GRADESCOPE_ENV' not in os.environ:
  # reset the parameters to initial random value
  model.reset_parameters()

  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
  loss_fn = F.nll_loss

  best_model = None
  best_valid_acc = 0

  for epoch in range(1, 1 + args["epochs"]):
    loss = train(model, data, train_idx, optimizer, loss_fn)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

### Question 5 ：你的**最佳模型**验证集和测试集精度如何？

运行下面的代码单元格，可以查看你最优模型的预测结果，  
并将模型的预测保存到名为 `ogbn-arxiv_node.csv` 的文件中。

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  best_result = test(best_model, data, split_idx, evaluator, save_model_results=True)
  train_acc, valid_acc, test_acc = best_result
  print(f'Best model: '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

# 4） GNN：图性质预测

在这一节中我们将创建一个为图性质预测的 GNN

## 加载并预处理数据集

In [ ]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm import tqdm

if 'IS_GRADESCOPE_ENV' not in os.environ:
  # 加载数据集
  dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print('Device: {}'.format(device))

  split_idx = dataset.get_idx_split()

  # 检查任务类型
  print('Task type: {}'.format(dataset.task_type))

In [ ]:
# 将数据集划分加载到对应的 dataloader 中
# 我们将在每批 32 个图上进行图分类任务的训练
# 对训练集中的图顺序进行打乱
if 'IS_GRADESCOPE_ENV' not in os.environ:
  train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
  valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
  test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  # Please do not change the args
  args = {
      'device': device,
      'num_layers': 5,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.001,
      'epochs': 30,
  }
  args

## 图预测模型

图的 Mini-Batching（小批量处理）

在正式进入模型之前，我们先介绍图数据的 mini-batching 概念。为了并行处理一小批图，  
PyG 会将这些图组合成一个**不相连的大图**数据对象（`torch_geometric.data.Batch`）。

`torch_geometric.data.Batch` 继承自之前介绍的 `torch_geometric.data.Data`，  
并额外包含一个名为 `batch` 的属性。

这个 `batch` 属性是一个向量，用来将每个节点映射到它在 mini-batch 中所属图的索引。例如：
<code>batch = [0, ..., 0, 1, ..., 1, ..., n - 2, n - 1, ..., n - 1]<code>

这个属性非常重要，它能帮助我们知道每个节点属于哪个图。  
举个例子，它可以用来对每个图的节点嵌入进行平均，从而得到图级别的嵌入表示。

### 补全

In [ ]:
现在，我们已经具备了实现 GCN 图预测模型所需的所有工具！

我们将复用现有的 GCN 模型来生成 **节点嵌入（node_embeddings）**，  
然后对节点进行 **全局池化（Global Pooling）**，从而得到每个图的**图级别嵌入（graph level embeddings）**，  
这些嵌入将用于预测每个图的属性。

请记住，`batch` 属性对于在 mini-batch 中执行全局池化操作是很重要的。

In [ ]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN 用于预测图属性
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # 加载分子图中原子的编码器
        self.node_encoder = AtomEncoder(hidden_dim)

        # 节点嵌入模型
        # 注意：输入维度和输出维度都设置为 hidden_dim
        self.gnn_node = GCN(hidden_dim, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=True)

        self.pool = None

        ############# Your code here ############
        ## 注意：
        ## 1. 将 self.pool 初始化为全局平均池化层（global mean pooling）
        ##    更多信息请参考文档：
        ##    https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers

        #########################################

        # 输出层
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        # TODO: 实现一个函数，输入是一批图（torch_geometric.data.Batch），
        # 返回的是每个图的预测属性。
        #
        # 注意：由于我们预测的是图级别的属性，
        # 输出张量的维度应该等于 mini-batch 中

In [ ]:
def train(model, device, data_loader, optimizer, loss_fn):
    # TODO: 实现一个使用给定优化器和损失函数训练模型的函数。
    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        ## 在计算训练损失时忽略包含 nan 的目标（未标注样本）
        is_labeled = batch.y == batch.y

        ############# Your code here ############
        ## 注意：
        ## 1. 对优化器执行 zero grad（清除梯度）
        ## 2. 将数据输入模型
        ## 3. 使用 `is_labeled` 掩码过滤输出和标签
        ## 4. 你可能需要将标签的类型转为 torch.float32
        ## 5. 将输出和标签传入 loss_fn 计算损失
        ## （大约 3 行代码）

        #########################################

        loss.backward()
        optimizer.step()

    return loss.item()


In [ ]:
# 用于分析的函数
def eval(model, device, loader, evaluator, save_model_results=False, save_file=None):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    if save_model_results:
        print ("Saving Model Predictions")

        # 创建一个包含两列的 pandas 数据框（DataFrame）
        # y_pred | y_true
        data = {}
        data['y_pred'] = y_pred.reshape(-1)
        data['y_true'] = y_true.reshape(-1)

        df = pd.DataFrame(data=data)
        # Save to csv
        df.to_csv('ogbg-molhiv_graph_' + save_file + '.csv', sep=',', index=False)

    return evaluator.eval(input_dict)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  model = GCN_Graph(args['hidden_dim'],
              dataset.num_tasks, args['num_layers'],
              args['dropout']).to(device)
  evaluator = Evaluator(name='ogbg-molhiv')

In [ ]:
import copy

if 'IS_GRADESCOPE_ENV' not in os.environ:
  model.reset_parameters()

  optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
  loss_fn = torch.nn.BCEWithLogitsLoss()

  best_model = None
  best_valid_acc = 0

  for epoch in range(1, 1 + args["epochs"]):
    print('Training...')
    loss = train(model, device, train_loader, optimizer, loss_fn)

    print('Evaluating...')
    train_result = eval(model, device, train_loader, evaluator)
    val_result = eval(model, device, valid_loader, evaluator)
    test_result = eval(model, device, test_loader, evaluator)

    train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
        best_model = copy.deepcopy(model)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')

### Quesion 6： 你的最佳模型的验证/测试 ROC-AUC 分数多少？

运行下方的代码单元格，以查看你最优模型的预测结果，  
并将预测分别保存为两个文件：`ogbg-molhiv_graph_valid.csv` 和 `ogbg-molhiv_graph_test.csv`。

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  train_auroc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
  valid_auroc = eval(best_model, device, valid_loader, evaluator, save_model_results=True, save_file="valid")[dataset.eval_metric]
  test_auroc  = eval(best_model, device, test_loader, evaluator, save_model_results=True, save_file="test")[dataset.eval_metric]

  print(f'Best model: '
      f'Train: {100 * train_auroc:.2f}%, '
      f'Valid: {100 * valid_auroc:.2f}% '
      f'Test: {100 * test_auroc:.2f}%')

### Question 7（选做）：在PyG中测试另外两种 global pooling

In [ ]:
############# Your code here ############